In [220]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [221]:
# Read in the data and display a preview
df = pd.read_csv('../data/ufc-master.csv')
df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
0,Thiago Santos,Johnny Walker,-150.0,130,66.666667,130.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,...,NaN,5.0,5:00,1500.0,800.0,900.0,2000.0,1600.0,-110.0,175.0
1,Alex Oliveira,Niko Price,170.0,-200,170.000000,50.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,450.0,350.0,700.0,1100.0,550.0,120.0
2,Misha Cirkunov,Krzysztof Jotko,110.0,-130,110.000000,76.923077,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,550.0,275.0,275.0,1400.0,600.0,185.0
3,Alexander Hernandez,Mike Breeden,-675.0,475,14.814815,475.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,...,Punch,1.0,1:20,80.0,175.0,900.0,500.0,3500.0,110.0,1100.0
4,Joe Solecki,Jared Gordon,-135.0,115,74.074074,115.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,165.0,200.0,400.0,1200.0,900.0,600.0


In [222]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4896 entries, 0 to 4895
Data columns (total 119 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   R_fighter                     object 
 1   B_fighter                     object 
 2   R_odds                        float64
 3   B_odds                        int64  
 4   R_ev                          float64
 5   B_ev                          float64
 6   date                          object 
 7   location                      object 
 8   country                       object 
 9   Winner                        object 
 10  title_bout                    bool   
 11  weight_class                  object 
 12  gender                        object 
 13  no_of_rounds                  int64  
 14  B_current_lose_streak         int64  
 15  B_current_win_streak          int64  
 16  B_draw                        int64  
 17  B_avg_SIG_STR_landed          float64
 18  B_avg_SIG_STR_pct          

I want to change date column to a datetime object but they are in different formats

In [223]:
df['date']

0       2021-10-02
1       2021-10-02
2       2021-10-02
3       2021-10-02
4       2021-10-02
           ...    
4891     3/21/2010
4892     3/21/2010
4893     3/21/2010
4894     3/21/2010
4895     3/21/2010
Name: date, Length: 4896, dtype: object

In [224]:
# Changing all / to - in date column
df['date'] = df['date'].str.replace('/', '-')

The first 58 dates are in a different format. They have year month day instead of month day year.

In [225]:
df['date'][:59]

0     2021-10-02
1     2021-10-02
2     2021-10-02
3     2021-10-02
4     2021-10-02
5     2021-10-02
6     2021-10-02
7     2021-10-02
8     2021-10-02
9     2021-10-02
10    2021-10-02
11    2021-09-25
12    2021-09-25
13    2021-09-25
14    2021-09-25
15    2021-09-25
16    2021-09-25
17    2021-09-25
18    2021-09-25
19    2021-09-25
20    2021-09-25
21    2021-09-25
22    2021-09-25
23    2021-09-25
24    2021-09-18
25    2021-09-18
26    2021-09-18
27    2021-09-18
28    2021-09-18
29    2021-09-18
30    2021-09-18
31    2021-09-18
32    2021-09-18
33    2021-09-18
34    2021-09-18
35    2021-09-18
36    2021-09-18
37    2021-09-04
38    2021-09-04
39    2021-09-04
40    2021-09-04
41    2021-09-04
42    2021-09-04
43    2021-09-04
44    2021-09-04
45    2021-09-04
46    2021-08-28
47    2021-08-28
48    2021-08-28
49    2021-08-28
50    2021-08-28
51    2021-08-28
52    2021-08-28
53    2021-08-28
54    2021-08-28
55    2021-08-28
56    2021-08-28
57    2021-08-28
58     8-21-20

Moving the year to end to have all dates in same format

In [226]:
df['date'][0][8:]

'02'

In [227]:
# Only want to change the first 58 values
for x in range(58):
    # Accessing each value in date column
    string = df.loc[x, 'date']
    # Slicing to get day, month, and year separated
    day = string[8:]
    month = string[5:7]
    year = string[:4]
    # Moving components around to get month-day-year format
    new_format = month + '-' + day + '-' + year
    # Update the DataFrame
    df.loc[x, 'date'] = new_format

In [228]:
df['date'][:59]

0     10-02-2021
1     10-02-2021
2     10-02-2021
3     10-02-2021
4     10-02-2021
5     10-02-2021
6     10-02-2021
7     10-02-2021
8     10-02-2021
9     10-02-2021
10    10-02-2021
11    09-25-2021
12    09-25-2021
13    09-25-2021
14    09-25-2021
15    09-25-2021
16    09-25-2021
17    09-25-2021
18    09-25-2021
19    09-25-2021
20    09-25-2021
21    09-25-2021
22    09-25-2021
23    09-25-2021
24    09-18-2021
25    09-18-2021
26    09-18-2021
27    09-18-2021
28    09-18-2021
29    09-18-2021
30    09-18-2021
31    09-18-2021
32    09-18-2021
33    09-18-2021
34    09-18-2021
35    09-18-2021
36    09-18-2021
37    09-04-2021
38    09-04-2021
39    09-04-2021
40    09-04-2021
41    09-04-2021
42    09-04-2021
43    09-04-2021
44    09-04-2021
45    09-04-2021
46    08-28-2021
47    08-28-2021
48    08-28-2021
49    08-28-2021
50    08-28-2021
51    08-28-2021
52    08-28-2021
53    08-28-2021
54    08-28-2021
55    08-28-2021
56    08-28-2021
57    08-28-2021
58     8-21-20

In [229]:
# Converting column to datetime from object
df['date'] = pd.to_datetime(df['date'], format='%m-%d-%Y')

In [230]:
df['date']

0      2021-10-02
1      2021-10-02
2      2021-10-02
3      2021-10-02
4      2021-10-02
          ...    
4891   2010-03-21
4892   2010-03-21
4893   2010-03-21
4894   2010-03-21
4895   2010-03-21
Name: date, Length: 4896, dtype: datetime64[ns]

In [231]:
df.describe()

,R_odds,B_odds,R_ev,B_ev,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,...,B_Flyweight_rank,B_Pound-for-Pound_rank,finish_round,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
count,4895.000000,4896.000000,4895.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,3966.000000,4131.000000,...,95.000000,35.000000,4274.000000,4274.000000,4093.000000,4077.000000,3847.000000,3835.000000,3847.000000,3834.000000
mean,-117.640449,66.030637,94.827397,167.083323,3.181985,0.477941,0.875408,0.010621,26.308553,0.444741,...,8.473684,9.485714,2.408049,652.313758,294.064745,416.544027,843.010138,1064.543155,514.231869,647.257173
std,268.881452,247.803928,82.843409,136.944643,0.571515,0.769386,1.311379,0.108333,20.935885,0.121332,...,4.259763,4.300283,0.996643,357.911423,230.583958,306.571299,550.126761,627.285034,413.622768,458.846643
min,-1700.000000,-1200.000000,5.882353,8.333333,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,1.000000,5.000000,-440.000000,-200.000000,-370.000000,-1250.000000,-550.000000,-275.000000
25%,-255.000000,-145.000000,39.215686,68.965517,3.000000,0.000000,0.000000,0.000000,5.610000,0.387500,...,5.000000,5.000000,1.000000,297.000000,167.000000,225.000000,435.000000,590.000000,240.000000,325.000000
50%,-150.000000,130.000000,66.666667,130.000000,3.000000,0.000000,0.000000,0.000000,24.759615,0.450000,...,8.000000,10.000000,3.000000,900.000000,250.000000,349.000000,720.000000,975.000000,435.000000,548.500000
75%,126.500000,220.000000,126.500000,220.000000,3.000000,1.000000,1.000000,0.000000,39.075000,0.510000,...,12.000000,13.500000,3.000000,900.000000,400.000000,525.000000,1200.000000,1400.000000,700.000000,880.750000
max,775.000000,1300.000000,775.000000,1300.000000,5.000000,6.000000,12.000000,2.000000,154.000000,1.000000,...,15.000000,15.000000,5.000000,1500.000000,2200.000000,2600.000000,4665.000000,4785.000000,2675.000000,3200.000000


In [232]:
df.loc[df['R_odds'].isna()]

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
32,Rong Zhu,Brandon Jenkins,NaN,195,NaN,195.0,2021-09-18,"Las Vegas, Nevada, USA",USA,Red,...,Punches,3.0,4:35,875.0,NaN,500.0,NaN,1600.0,NaN,450.0


In [233]:
df.isna().sum()

R_fighter        0
B_fighter        0
R_odds           1
B_odds           0
R_ev             1
              ... 
b_dec_odds     819
r_sub_odds    1049
b_sub_odds    1061
r_ko_odds     1049
b_ko_odds     1062
Length: 119, dtype: int64

In [234]:
nulls = df.isna().sum()
nulls.sort_values(ascending=False)[:30]

B_Women's Featherweight_rank    4896
R_Women's Featherweight_rank    4889
B_Pound-for-Pound_rank          4861
B_Women's Flyweight_rank        4852
R_Women's Flyweight_rank        4837
B_Women's Strawweight_rank      4835
B_Women's Bantamweight_rank     4818
B_Bantamweight_rank             4811
B_Lightweight_rank              4809
B_Welterweight_rank             4807
B_Featherweight_rank            4806
B_Light Heavyweight_rank        4803
B_Flyweight_rank                4801
B_Middleweight_rank             4794
R_Women's Strawweight_rank      4792
B_Heavyweight_rank              4786
R_Women's Bantamweight_rank     4778
R_Featherweight_rank            4763
R_Middleweight_rank             4762
R_Bantamweight_rank             4759
R_Lightweight_rank              4757
R_Welterweight_rank             4756
R_Light Heavyweight_rank        4755
R_Heavyweight_rank              4754
R_Flyweight_rank                4754
R_Pound-for-Pound_rank          4730
B_match_weightclass_rank        4019
R

In [235]:
columns_to_drop = list(nulls.sort_values(ascending=False)[:29].index)
columns_to_drop

["B_Women's Featherweight_rank",
 "R_Women's Featherweight_rank",
 'B_Pound-for-Pound_rank',
 "B_Women's Flyweight_rank",
 "R_Women's Flyweight_rank",
 "B_Women's Strawweight_rank",
 "B_Women's Bantamweight_rank",
 'B_Bantamweight_rank',
 'B_Lightweight_rank',
 'B_Welterweight_rank',
 'B_Featherweight_rank',
 'B_Light Heavyweight_rank',
 'B_Flyweight_rank',
 'B_Middleweight_rank',
 "R_Women's Strawweight_rank",
 'B_Heavyweight_rank',
 "R_Women's Bantamweight_rank",
 'R_Featherweight_rank',
 'R_Middleweight_rank',
 'R_Bantamweight_rank',
 'R_Lightweight_rank',
 'R_Welterweight_rank',
 'R_Light Heavyweight_rank',
 'R_Heavyweight_rank',
 'R_Flyweight_rank',
 'R_Pound-for-Pound_rank',
 'B_match_weightclass_rank',
 'R_match_weightclass_rank',
 'finish_details']

In [236]:
df.drop(columns_to_drop, axis=1, inplace=True)

In [237]:
nulls2 = df.isna().sum()
nulls2.sort_values(ascending=False)

b_ko_odds                      1062
b_sub_odds                     1061
r_ko_odds                      1049
r_sub_odds                     1049
B_avg_SIG_STR_landed            930
                               ... 
R_win_by_Decision_Split           0
R_win_by_Decision_Unanimous       0
R_win_by_KO/TKO                   0
R_win_by_Submission               0
R_fighter                         0
Length: 90, dtype: int64

In [239]:
clean_df = df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2615 entries, 0 to 4286
Data columns (total 90 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   R_fighter                     2615 non-null   object        
 1   B_fighter                     2615 non-null   object        
 2   R_odds                        2615 non-null   float64       
 3   B_odds                        2615 non-null   int64         
 4   R_ev                          2615 non-null   float64       
 5   B_ev                          2615 non-null   float64       
 6   date                          2615 non-null   datetime64[ns]
 7   location                      2615 non-null   object        
 8   country                       2615 non-null   object        
 9   Winner                        2615 non-null   object        
 10  title_bout                    2615 non-null   bool          
 11  weight_class                  

Timestamp('2012-05-05 00:00:00')